# JSON PROCESSING

### LIBRARIES

In [1]:
import pandas as pd
import os
import json
from datetime import datetime

### FUNCTIONS

In [36]:
def flatten_categories(data):
    categories_info = data.get('categories', [])
    if categories_info:
        first_category_name = categories_info[0]['name']
    else:
        first_category_name = None
    return first_category_name

def flatten_products(data):
    flattened_data = []
    for product in data['products']:
        flattened_product = product.copy()
        
        # Flatten badges and price_instructions
        for subfield in ['badges', 'price_instructions']:
            subfield_data = flattened_product.pop(subfield, None)
            if subfield_data:
                if isinstance(subfield_data, dict):
                    for key, value in subfield_data.items():
                        flattened_product[f"{subfield}_{key}"] = value
                else:
                    flattened_product[f"{subfield}"] = subfield_data
        
        # Flatten categories
        categories_data = flattened_product.pop('categories', None)
        if categories_data:
            categories_ids = [category['id'] for category in categories_data]
            categories_names = [category['name'] for category in categories_data]
            flattened_product['categories_id'] = ', '.join(map(str, categories_ids))
            flattened_product['categories_name'] = ', '.join(categories_names)

        flattened_data.append(flattened_product)
        
    return flattened_data

def process_json(data):
    all_products = []
    sub_category_id = data['id']  # Get the subcategory ID
    sub_category_name = data['name']  # Get the subcategory name
    
    for category in data.get('categories', []):
        category_id = category['id']  # Get the category ID
        category_name = category['name']
        for product_info in category.get('products', []):
            flattened_product_info = flatten_products({'products': [product_info]})
            for product in flattened_product_info:
                product['third_category_name'] = category_name
                product['third_category_id'] = category_id  # Add the category ID
                product['sub_category_id'] = sub_category_id  # Add the subcategory ID
                product['sub_category_name'] = sub_category_name  # Add the subcategory name
                all_products.append(product)
    
    df = pd.DataFrame(all_products)
    return df

### CREATING DFs AND CSVs

In [37]:
# Directory where JSON files are located
json_dir = "jsons_mercadona"
output_dir = "CSV"

In [38]:
# Get the list of JSON files in the directory
json_files = [file for file in os.listdir(json_dir) if file.startswith("data_")]

In [ ]:
# Get the current date
current_date = datetime.now().strftime("%Y-%m-%d")

In [40]:
# Iterate over each JSON file
for json_file in json_files:
    
    # Get the subcategory ID from the file name
    sub_category_id = int(json_file.split("_")[1].split(".")[0])
    
    # Load the JSON file with 'utf-8' encoding
    with open(os.path.join(json_dir, json_file), 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    # Process the JSON file
    df = process_json(data)
    
    # Drop specified columns
    columns_to_drop = ['limit', 'share_url', 'thumbnail', 'unavailable_from', 'unavailable_weekdays']
    df.drop(columns=columns_to_drop, inplace=True)
    
    # Extract the value from the "categories_id" column (assuming it's unique for all records)
    categories_id = df['categories_id'].iloc[0]
    
    # Construct the CSV file name with the value of "categories_id", sub_category_id, and the current date
    csv_filename = f"data_{categories_id}_{sub_category_id}_{current_date}.csv"
    csv_filepath = os.path.join(output_dir, csv_filename)
    
    # Save the DataFrame as CSV in the output directory
    df.to_csv(csv_filepath, index=False)
    print(f"Archivo CSV guardado en: {csv_filepath}")

Archivo CSV guardado en: CSV\data_10_100_2024-03-24.csv
Archivo CSV guardado en: CSV\data_11_103_2024-03-24.csv
Archivo CSV guardado en: CSV\data_11_104_2024-03-24.csv
Archivo CSV guardado en: CSV\data_11_105_2024-03-24.csv
Archivo CSV guardado en: CSV\data_11_106_2024-03-24.csv
Archivo CSV guardado en: CSV\data_11_107_2024-03-24.csv
Archivo CSV guardado en: CSV\data_11_108_2024-03-24.csv
Archivo CSV guardado en: CSV\data_11_109_2024-03-24.csv
Archivo CSV guardado en: CSV\data_11_110_2024-03-24.csv
Archivo CSV guardado en: CSV\data_11_111_2024-03-24.csv
Archivo CSV guardado en: CSV\data_12_112_2024-03-24.csv
Archivo CSV guardado en: CSV\data_12_115_2024-03-24.csv
Archivo CSV guardado en: CSV\data_12_116_2024-03-24.csv
Archivo CSV guardado en: CSV\data_12_117_2024-03-24.csv
Archivo CSV guardado en: CSV\data_13_118_2024-03-24.csv
Archivo CSV guardado en: CSV\data_13_120_2024-03-24.csv
Archivo CSV guardado en: CSV\data_13_121_2024-03-24.csv
Archivo CSV guardado en: CSV\data_14_122_2024-03

### FINAL CSV

In [24]:
# Folder containing CSV files
carpeta = 'CSV'

In [25]:
# List to store DataFrames from each CSV file
dfs = []

In [26]:
# Iterate over all files in the folder
for archivo in os.listdir(carpeta):
    if archivo.endswith('.csv'):  # Take only files with .csv extension
        # Read the CSV file and append it to the list of DataFrames
        ruta_archivo = os.path.join(carpeta, archivo)
        df = pd.read_csv(ruta_archivo)
        dfs.append(df)

In [27]:
# Combine all DataFrames into one
Mercadona_Productos = pd.concat(dfs, ignore_index=True)

In [28]:
Mercadona_Productos["id"] = Mercadona_Productos["id"].astype("int64")

In [29]:
Mercadona_Productos.to_csv("Mercadona_Productos.csv", index=False)


In [30]:
Mercadona_Productos

,id,slug,packaging,published,display_name,badges_is_water,badges_requires_age_check,price_instructions_iva,price_instructions_is_new,price_instructions_is_pack,...,price_instructions_min_bunch_amount,price_instructions_reference_format,price_instructions_previous_unit_price,price_instructions_increment_bunch_amount,categories_id,categories_name,third_category_name,third_category_id,sub_category_id,sub_category_name
0,86039,zumo-melocoton-uva-hacendado-brick,Brick,True,Zumo de melocotón y uva Hacendado,False,False,10,False,False,...,1.0,L,1.1,1.0,10,Zumos,Melocotón,398,100,Melocotón y piña
1,86040,zumo-melocoton-uva-hacendado-pack-6,Pack-6,True,Zumo de melocotón y uva Hacendado,False,False,10,False,True,...,1.0,L,NaN,1.0,10,Zumos,Melocotón,398,100,Melocotón y piña
2,39219,bebida-melocoton-hacendado-sin-azucares-anadid...,Brick,True,Bebida de melocotón Hacendado sin azúcares aña...,False,False,21,False,False,...,1.0,L,NaN,1.0,10,Zumos,Melocotón,398,100,Melocotón y piña
3,39226,bebida-melocoton-hacendado-sin-azucares-anadid...,Pack-6,True,Bebida de melocotón Hacendado sin azúcares aña...,False,False,21,False,True,...,1.0,L,NaN,1.0,10,Zumos,Melocotón,398,100,Melocotón y piña
4,39129,zumo-pina-uva-hacendado-brick,Brick,True,Zumo de piña y uva Hacendado,False,False,10,False,False,...,1.0,L,NaN,1.0,10,Zumos,Piña,397,100,Melocotón y piña
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5147,60538,golosinas-mix-pascua-hacendado-paquete,Paquete,True,Golosinas Mix Pascua Hacendado,False,False,10,False,False,...,1.0,kg,NaN,1.0,9,"Azúcar, caramelos y chocolate",Golosinas,395,97,Golosinas
5148,60539,caramelos-rellenos-pascua-soft-eggies-trolli-p...,Paquete,True,Caramelos rellenos Pascua Soft Eggies Trolli,False,False,10,False,False,...,1.0,kg,NaN,1.0,9,"Azúcar, caramelos y chocolate",Golosinas,395,97,Golosinas
5149,60567,golosinas-rega-fruit-mix-hacendado-paquete,Paquete,True,Golosinas Rega Fruit Mix Hacendado,False,False,10,False,False,...,1.0,kg,NaN,1.0,9,"Azúcar, caramelos y chocolate",Golosinas,395,97,Golosinas
5150,60631,golosinas-fresi-cream-hacendado-merengue-recub...,Paquete,True,Golosinas Fresi Cream Hacendado merengue recub...,False,False,10,False,False,...,1.0,kg,NaN,1.0,9,"Azúcar, caramelos y chocolate",Golosinas,395,97,Golosinas
